In [2]:
!export HF_HOME='./.hf'
!export TRANSFORMERS_CACHE='./.hf'
!export TRANSFORMERS_HOME='./.hf'
!export HF_CACHE='./.hf'

import os
os.environ['HF_HOME'] = './.hf/'


import sys
import json
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

import pandas as pd

file_path = "questions-de.tsv"
records = pd.read_csv(file_path, sep='\t').to_dict(orient='records')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [3]:
pd.read_csv(file_path, sep='\t')

,Primary Wikipedia URL,title,URL-de,question
0,https://en.wikipedia.org/wiki/David_F._Sandberg,David F. Sandberg,https://de.wikipedia.org/wiki/David_F._Sandberg,Welche Filme produzierte David Sandberg im Jah...
1,https://en.wikipedia.org/wiki/Mouthier-Haute-P...,Mouthier-Haute-Pierre,https://de.wikipedia.org/wiki/Mouthier-Haute-P...,Wann wurde das Benediktinerklosters Saint-Pier...
2,https://en.wikipedia.org/wiki/Cilleruelo_de_Sa...,Cilleruelo de San Mamés,https://de.wikipedia.org/wiki/Cilleruelo_de_Sa...,In welcher Provinz liegt die spanische Gemeind...
3,https://en.wikipedia.org/wiki/Helsingin_Jalkap...,Helsingin Jalkapalloklubi,https://de.wikipedia.org/wiki/HJK_Helsinki,Zu welchem Verein wechselte Dawda Bah im Jahr ...
4,https://en.wikipedia.org/wiki/Gampel-Bratsch,Gampel-Bratsch,https://de.wikipedia.org/wiki/Gampel-Bratsch,"Welchen Presi gewann der Dokumentafilm ""Bratsc..."
...,...,...,...,...
195,https://zh.wikipedia.org/wiki/%E5%90%89%E5%88%...,吉列体育场,https://de.wikipedia.org/wiki/Gillette_Stadium,Wie viele Zuschauer haben Platz im Gillette St...
196,https://fr.wikipedia.org/wiki/Oxycarenidae,Oxycarenidae,https://de.wikipedia.org/wiki/Oxycarenidae,Wie viele Gattungen umfasst die Familie der Ox...
197,https://sv.wikipedia.org/wiki/Microsoft_Edge,Microsoft Edge,https://de.wikipedia.org/wiki/Microsoft_Edge,Seit wann kann man Microsoft Edge auf iOS und ...
198,https://en.wikipedia.org/wiki/Bombykol,Bombykol,https://de.wikipedia.org/wiki/Bombykol,Wozu benutzen Insekten Bombykol?


In [4]:
configs = [
    ('k50_p0.90_t0.1', dict(top_k=50, top_p=0.90, temperature=0.1)),
    ('k50_p0.95_t0.1', dict(top_k=50, top_p=0.95, temperature=0.1)),
    ('k50_p0.90_t0.2', dict(top_k=50, top_p=0.90, temperature=0.2)),
    ('k50_p0.95_t0.2', dict(top_k=50, top_p=0.95, temperature=0.2)),
    ('k50_p0.90_t0.3', dict(top_k=50, top_p=0.90, temperature=0.3)),
    ('k50_p0.95_t0.3', dict(top_k=50, top_p=0.95, temperature=0.3)),
    ('k75_p0.90_t0.1', dict(top_k=75, top_p=0.90, temperature=0.1)),
    ('k75_p0.95_t0.1', dict(top_k=75, top_p=0.95, temperature=0.1)),
    ('k75_p0.90_t0.2', dict(top_k=75, top_p=0.90, temperature=0.2)),
    ('k75_p0.95_t0.2', dict(top_k=75, top_p=0.95, temperature=0.2)),
    ('k75_p0.90_t0.3', dict(top_k=75, top_p=0.90, temperature=0.3)),
    ('k75_p0.95_t0.3', dict(top_k=75, top_p=0.95, temperature=0.3)),
]


In [5]:
# model_name = 'togethercomputer/Pythia-Chat-Base-7B' 
model_name = 'malteos/bloom-6b4-clp-german-oasst-v0.1' 

configs = [
    ('k50_p0.90_t0.1', dict(top_k=50, top_p=0.90, temperature=0.1)),
    ('k50_p0.95_t0.1', dict(top_k=50, top_p=0.95, temperature=0.1)),
    ('k50_p0.90_t0.2', dict(top_k=50, top_p=0.90, temperature=0.2)),
    ('k50_p0.95_t0.2', dict(top_k=50, top_p=0.95, temperature=0.2)),
    ('k50_p0.90_t0.3', dict(top_k=50, top_p=0.90, temperature=0.3)),
    ('k50_p0.95_t0.3', dict(top_k=50, top_p=0.95, temperature=0.3)),
    ('k75_p0.90_t0.1', dict(top_k=75, top_p=0.90, temperature=0.1)),
    ('k75_p0.95_t0.1', dict(top_k=75, top_p=0.95, temperature=0.1)),
    ('k75_p0.90_t0.2', dict(top_k=75, top_p=0.90, temperature=0.2)),
    ('k75_p0.95_t0.2', dict(top_k=75, top_p=0.95, temperature=0.2)),
    ('k75_p0.90_t0.3', dict(top_k=75, top_p=0.90, temperature=0.3)),
    ('k75_p0.95_t0.3', dict(top_k=75, top_p=0.95, temperature=0.3)),
]

model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import tqdm.notebook as tqdm
from transformers.utils import logging
import pathlib
logging.set_verbosity_warning()

for shorthand, config in tqdm.tqdm(configs):
    output_file_path = f'german-{model_name.split("/")[1]}.{shorthand}.jsonl'
    anootation_file_path = f'german-{model_name.split("/")[1]}-annotation.{shorthand}.jsonl'
    if not pathlib.Path(anootation_file_path).is_file():
        with open(output_file_path, 'w', encoding='utf-8') as file:
            for record in tqdm.tqdm(records):
                message = record['question']
                # prompt = f"<human>: {message}\n<bot>:"
                prompt = message + '\n'
                inputs = tokenizer(prompt, return_tensors="pt").to(device)
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=512,
                    num_return_sequences=1,
                    no_repeat_ngram_size=2,
                    return_dict_in_generate=True,
                    output_logits=True,
                    do_sample=True,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id,
                    # eos_token_id=tokenizer.encode('\n'),
                    # pad_token_id=tokenizer.encode('\n')[0],
                    **config,
                )
        
                response_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
                response_text = response_text.replace(prompt, "") # response repeats the input in the begining
                response_token_ids = outputs.sequences[0].to("cpu").tolist()[len(inputs.input_ids[0]):]
                # response_embeddings = outputs.sequences[0].to("cpu").tolist()[len(inputs.input_ids[0]):]
                response_tokens = tokenizer.convert_ids_to_tokens(response_token_ids)
                response_logits = [l.to("cpu").tolist() for l in outputs.logits]
                
                """print("\n\n")
                print(f"Q: {message}")
                print(f"A: {response_text}")
        
                print("input length", len(inputs.input_ids[0]))
                # print("sequence length", len(outputs.sequences[0]))
                print("response token length", len(response_tokens))
                print("response token ID length", len(response_token_ids))
                print("logits length", len(response_logits))
                # print("embedding length", len(response_embeddings))
                raise"""
        
        
                record['model_id'] = model_name
                record['lang'] = 'DE'
        
                record['output_text'] = response_text
                record['output_tokens'] = response_tokens
                record['output_logits'] = response_logits
                # record['output_embeddings'] = response_embeddings
        
                json.dump(record, file, ensure_ascii=False)
                file.write('\n')
        
        columns_to_extract = ['URL-de', 'lang', 'question', 'model_id', 'output_text', 'output_tokens', 'title']
        
        output_data = []
        
        with open(anootation_file_path, 'w', encoding='utf-8') as file:
            for data in records:
                extracted_data = {key: data[key] for key in columns_to_extract if key in data}
        
                json.dump(extracted_data, file, ensure_ascii=False)
                file.write('\n')

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [5]:
del model

In [5]:
from transformers import AutoTokenizer, MistralForCausalLM, set_seed
model_name = "occiglot/occiglot-7b-de-en-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MistralForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
import tqdm.notebook as tqdm
from transformers.utils import logging
import pathlib
logging.set_verbosity_warning()
#model = model.to(device)

for shorthand, config in tqdm.tqdm(configs):
    output_file_path = f'german-{model_name.split("/")[1]}.{shorthand}.jsonl'
    anootation_file_path = f'german-{model_name.split("/")[1]}-annotation.{shorthand}.jsonl'
    if not pathlib.Path(anootation_file_path).is_file():
        with open(output_file_path, 'w', encoding='utf-8') as file:
            for record in tqdm.tqdm(records):
                messages = [
                   {"role": "system", 'content': 'You are a helpful assistant. Please give short and concise answers.'},
                   {"role": "user", "content": record['question']},
                ]
                inputs = tokenizer.apply_chat_template(
                    messages, 
                    tokenize=True, 
                    add_generation_prompt=True, 
                    return_dict=False, 
                    return_tensors='pt',
                ).to(device)
                outputs = model.generate(
                    inputs,
                    max_new_tokens=256,
                    num_return_sequences=1,
                    no_repeat_ngram_size=2,
                    return_dict_in_generate=True,
                    output_logits=True,
                    do_sample=True,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id,
                    # eos_token_id=tokenizer.encode('\n'),
                    # pad_token_id=tokenizer.encode('\n')[0],
                    **config,
                )
        
                response_text = tokenizer.decode(outputs.sequences[0][len(inputs[0]):], skip_special_tokens=True)
                # response_text = response_text.replace(prompt, "") # response repeats the input in the begining
                response_token_ids = outputs.sequences[0].to("cpu").tolist()[len(inputs[0]):]
                # response_embeddings = outputs.sequences[0].to("cpu").tolist()[len(inputs.input_ids[0]):]
                response_tokens = tokenizer.convert_ids_to_tokens(response_token_ids)
                response_logits = [l.to("cpu").tolist() for l in outputs.logits]
                
                """print("\n\n")
                print(f"Q: {message}")
                print(f"A: {response_text}")
        
                print("input length", len(inputs.input_ids[0]))
                # print("sequence length", len(outputs.sequences[0]))
                print("response token length", len(response_tokens))
                print("response token ID length", len(response_token_ids))
                print("logits length", len(response_logits))
                # print("embedding length", len(response_embeddings))
                raise"""
        
        
                record['model_id'] = model_name
                record['lang'] = 'DE'
        
                record['output_text'] = response_text
                record['output_tokens'] = response_tokens
                record['output_logits'] = response_logits
                # record['output_embeddings'] = response_embeddings
        
                json.dump(record, file, ensure_ascii=False)
                file.write('\n')
        
        columns_to_extract = ['URL-de', 'lang', 'question', 'model_id', 'output_text', 'output_tokens', 'title']
        
        output_data = []
        
        with open(anootation_file_path, 'w', encoding='utf-8') as file:
            for data in records:
                extracted_data = {key: data[key] for key in columns_to_extract if key in data}
        
                json.dump(extracted_data, file, ensure_ascii=False)
                file.write('\n')

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]